In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeuronNet(nn.Module):
    def __init__(self, num_classes):
        super(NeuronNet, self).__init__()
        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.shortcut1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=1),
            nn.BatchNorm2d(64)
        )

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=2, dilation=2),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=2, dilation=2),
            nn.BatchNorm2d(128),
            nn.ReLU()
        )
        self.shortcut2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=1),
            nn.BatchNorm2d(128)
        )

        # Global Average Pooling
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully Connected Layer
        self.fc = nn.Linear(128, num_classes)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Block 1
        identity = x
        out = self.block1(x)
        out = out + self.shortcut1(identity)
        out = F.relu(out)

        # Block 2
        identity = out
        out = self.block2(out)
        out = out + self.shortcut2(identity)
        out = F.relu(out)

        # Global Average Pooling
        out = self.global_pool(out)
        out = out.view(out.size(0), -1)

        # Dropout
        out = self.dropout(out)

        # Fully Connected Layer
        out = self.fc(out)
        return out

# Create PyTorch model and load state
num_classes = 10
model = NeuronNet(num_classes)

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 0.3M


In [11]:
torch.save(model.state_dict(), 'neuron_net.pth')

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models

def NeuronNet_tf(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))

    # Block 1
    x = layers.Conv2D(64, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    shortcut = layers.Conv2D(64, (1, 1), strides=1)(inputs)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)

    # Block 2
    identity = x
    x = layers.Conv2D(128, (3, 3), padding='same', dilation_rate=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', dilation_rate=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    shortcut = layers.Conv2D(128, (1, 1), strides=1)(identity)
    shortcut = layers.BatchNormalization()(shortcut)

    x = layers.add([x, shortcut])
    x = layers.ReLU()(x)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Dropout
    x = layers.Dropout(0.5)(x)

    # Fully Connected Layer
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

num_classes = 10
tf_model = NeuronNet_tf(num_classes)

n_parameters = tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 0.3M


In [13]:
pytorch_model = NeuronNet(num_classes)

pytorch_model.load_state_dict(torch.load('/kaggle/working/neuron_net.pth'))
pytorch_model.eval()

# Helper function to convert PyTorch tensors to NumPy arrays
def pt_to_np(tensor):
    return tensor.detach().cpu().numpy()

In [14]:
# for i, layer in enumerate(tf_model.layers):
#     print(i, layer.name, [w.shape for w in layer.get_weights()])

In [15]:
# Map PyTorch weights to TensorFlow model based on correct indices
# Block 1
tf_model.layers[1].set_weights([pt_to_np(pytorch_model.block1[0].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.block1[0].bias)])
tf_model.layers[2].set_weights([pt_to_np(pytorch_model.block1[1].weight), pt_to_np(pytorch_model.block1[1].bias),
                                pt_to_np(pytorch_model.block1[1].running_mean), pt_to_np(pytorch_model.block1[1].running_var)])
tf_model.layers[4].set_weights([pt_to_np(pytorch_model.block1[3].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.block1[3].bias)])
tf_model.layers[5].set_weights([pt_to_np(pytorch_model.block1[4].weight), pt_to_np(pytorch_model.block1[4].bias),
                                pt_to_np(pytorch_model.block1[4].running_mean), pt_to_np(pytorch_model.block1[4].running_var)])
tf_model.layers[6].set_weights([pt_to_np(pytorch_model.shortcut1[0].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.shortcut1[0].bias)])
tf_model.layers[8].set_weights([pt_to_np(pytorch_model.shortcut1[1].weight), pt_to_np(pytorch_model.shortcut1[1].bias),
                                pt_to_np(pytorch_model.shortcut1[1].running_mean), pt_to_np(pytorch_model.shortcut1[1].running_var)])

# Block 2
tf_model.layers[11].set_weights([pt_to_np(pytorch_model.block2[0].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.block2[0].bias)])
tf_model.layers[12].set_weights([pt_to_np(pytorch_model.block2[1].weight), pt_to_np(pytorch_model.block2[1].bias),
                                 pt_to_np(pytorch_model.block2[1].running_mean), pt_to_np(pytorch_model.block2[1].running_var)])
tf_model.layers[14].set_weights([pt_to_np(pytorch_model.block2[3].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.block2[3].bias)])
tf_model.layers[15].set_weights([pt_to_np(pytorch_model.block2[4].weight), pt_to_np(pytorch_model.block2[4].bias),
                                 pt_to_np(pytorch_model.block2[4].running_mean), pt_to_np(pytorch_model.block2[4].running_var)])
tf_model.layers[16].set_weights([pt_to_np(pytorch_model.shortcut2[0].weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.shortcut2[0].bias)])
tf_model.layers[18].set_weights([pt_to_np(pytorch_model.shortcut2[1].weight), pt_to_np(pytorch_model.shortcut2[1].bias),
                                 pt_to_np(pytorch_model.shortcut2[1].running_mean), pt_to_np(pytorch_model.shortcut2[1].running_var)])

# Fully Connected Layer
tf_model.layers[23].set_weights([pt_to_np(pytorch_model.fc.weight.T), pt_to_np(pytorch_model.fc.bias)])

In [16]:
# Save the TensorFlow model
tf_model.save('neuron_net_tf.h5')

In [17]:
# Load the saved TensorFlow model
loaded_tf_model = tf.keras.models.load_model('neuron_net_tf.h5')

In [18]:
n_parameters = loaded_tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 0.3M
